# Preprocessing of the metabolic model of Sulfolobus solfataricus

Author: Diana Szeliova \
Last update: 15.7.2024

* Sulfolobus solfataricus model from https://doi.org/10.1111/mmi.13498 downloaded from https://fairdomhub.org/models/225
* rename metabolites and reactions so that they subscribe to cobrapy standards
* add missing reactions

In [1]:
import cobra
import re
import pickle
import pandas as pd
from cobra import Reaction

In [2]:
modelname = "Sulfolobus-solfataricus_D-glucose.xml"
model = cobra.io.sbml.validate_sbml_model(f"../data/{modelname}")

SBML errors in validation, check error log for details.
COBRA errors in validation, check error log for details.
COBRA warnings in validation, check error log for details.


### Find reaction and metabolite IDs in the SBML file

In [3]:
metabolite_ids = []
reaction_ids = []
with open(f"../data/{modelname}") as file:
    lines = file.readlines()

    # get lists of metabolite and reaction ids
    for line in lines:
        try:
            metabolite_ids.append(re.search('<species(.*)id="(.*)" name', line).group(2))
        except AttributeError:
            pass

        try:
            reaction_ids.append(re.search('<reaction id="(.*)" name', line).group(1))
        except AttributeError:
            pass

### Rename metabolite and reaction IDs to numbers

In [4]:
new_metabolite_ids = {f"M_{idx}": old_id for idx, old_id in enumerate(metabolite_ids)}
new_reaction_ids = {f"R_{idx}": old_id for idx, old_id in enumerate(reaction_ids)}

In [5]:
# replace old ids with new ids
new_lines = []
for line in lines:
    for new_id, old_id in new_metabolite_ids.items():
        if f'"{old_id}"' in line:
            line = line.replace(old_id, new_id)

    for new_id, old_id in new_reaction_ids.items():
        if f'"{old_id}"' in line:
            line = line.replace(old_id, new_id)

    if "<listOfProducts/>" in line or "<notes/>" in line:
        continue
    new_lines.append(line)

### Other corrections

In [6]:
new_lines[1] = '<sbml level="2" version="4" xmlns="http://www.sbml.org/sbml/level2/version4" xmlns:html="http://www.w3.org/1999/xhtml">'

In [7]:
new_lines[2] = new_lines[2].replace("Unnamed model - D-glucose", "Sulfolobus")

### Write the model, them read it again

In [8]:
new_modelname = "Sulfolobus-solfataricus_corrected"
with open(f"../data/{new_modelname}.xml", "w") as file:
    for line in new_lines:
        file.write(line)

In [9]:
model = cobra.io.sbml.validate_sbml_model(f"../data/{new_modelname}.xml")

COBRA warnings in validation, check error log for details.


In [10]:
model = cobra.io.read_sbml_model(f"../data/{new_modelname}.xml")

Model does not contain SBML fbc package information.
SBML package 'layout' not supported by cobrapy, information is not parsed
SBML package 'render' not supported by cobrapy, information is not parsed
Encoding LOWER_BOUND and UPPER_BOUND in KineticLaw is discouraged, use fbc:fluxBounds instead: <Reaction R_0 "R_0">
Encoding LOWER_BOUND and UPPER_BOUND in KineticLaw is discouraged, use fbc:fluxBounds instead: <Reaction R_1 "R_1">
Encoding LOWER_BOUND and UPPER_BOUND in KineticLaw is discouraged, use fbc:fluxBounds instead: <Reaction R_2 "R_2">
Encoding LOWER_BOUND and UPPER_BOUND in KineticLaw is discouraged, use fbc:fluxBounds instead: <Reaction R_3 "R_3">
Encoding LOWER_BOUND and UPPER_BOUND in KineticLaw is discouraged, use fbc:fluxBounds instead: <Reaction R_4 "R_4">
Missing upper flux bound set to '1000.0' for reaction: '<Reaction R_4 "R_4">'
Encoding LOWER_BOUND and UPPER_BOUND in KineticLaw is discouraged, use fbc:fluxBounds instead: <Reaction R_5 "R_5">
Missing upper flux bound 

### Other model changes
* add lysine exchange reaction

In [11]:
# add secretion for lysine
reaction = Reaction(f"{len(new_reaction_ids)}")
reaction.name = reaction.id
reaction.lower_bound = 0.
reaction.upper_bound = 1000.
reaction.add_metabolites({model.metabolites.get_by_id("10"): -1})
model.add_reactions([reaction])

In [12]:
reaction

Reaction identifier,992
Name,992
Memory address,0x7f3e64bb4d00
Stoichiometry,10 --> M_10 -->
GPR,
Lower bound,0.0
Upper bound,1000.0


In [13]:
new_reaction_ids[f"R_{reaction.id}"] = "T_flux_L-lysine"

In [14]:
with open('../data/sulfolobus_metabolite_ids.pkl', 'wb') as fp:
    pickle.dump(new_metabolite_ids, fp)

with open('../data/sulfolobus_reaction_ids.pkl', 'wb') as fp:
    pickle.dump(new_reaction_ids, fp)

## Rename metabolites and reactions (for plotting)

In [15]:
node_names = pd.read_csv("../data/node_names_escher.csv")
node_names = node_names.drop_duplicates()
node_names['node'] = node_names['node'].astype(str)
node_dict = dict(zip(node_names.node, node_names.short_name))

In [16]:
for metabolite in model.metabolites:
    try:
        metabolite.name = node_dict[metabolite.id]
    except KeyError:
        continue

In [17]:
for reaction in model.reactions:
    reaction.name = ""

### Write the model and test if it can be read without problems

In [18]:
# save as json for escher
cobra.io.save_json_model(model, f"../data/{new_modelname}.json")

In [19]:
cobra.io.write_sbml_model(model, f"../data/{new_modelname}.xml")

In [20]:
# test if model can be read without problems
model = cobra.io.read_sbml_model(f"../data/{new_modelname}.xml")

In [21]:
model

Name,Sulfolobus
Memory address,0x07f3e64b5fa90
Number of metabolites,865
Number of reactions,993
Number of groups,0
Objective expression,1.0*0 - 1.0*0_reverse_cfcd2
Compartments,"cytosol, extracellular"
